In [1]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import time
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Get data from csv file
data = pd.read_csv("Classification/heart.csv") 
# Preview the first 5 lines of the loaded data 
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
#Explore data
print ("Rows     : " , data.shape[0])
print ("Columns  : " , data.shape[1])
print ("\nFeatures : \n", data.columns.tolist())
print ("\nMissing values :  ", data.isnull().sum().values.sum())
print ("\nUnique values :  \n", data.nunique())

Rows     :  303
Columns  :  14

Features : 
 ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

Missing values :   0

Unique values :  
 age          41
sex           2
cp            4
trestbps     49
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            5
thal          4
target        2
dtype: int64


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [6]:
data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [9]:
#Change data types to flat32
data = data.astype('float32')

In [25]:
#normalize data and transform it to tensor
sc = StandardScaler()
X = sc.fit_transform(data[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal' ]].values)
X = torch.tensor(X, dtype=torch.float32)

Y = torch.tensor(data['target'].values, dtype=torch.float32)
Y = Y.view(Y.shape[0], 1)

In [32]:
#divide train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.70,test_size=0.30)

#Get data sample sizes
trainDataSize = X_train[:,0].size()
testDataSize = y_test[:,0].size()
modelParameters = X_train[0].size()
#modelParameters

torch.Size([13])

In [38]:
class LogisticReg(torch.nn.Module):
    def __init__(self, n_input):
        super(LogisticReg, self).__init__()
        self.linear = torch.nn.Linear(n_input, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.linear(x))
        return out

In [49]:
#train model
l_rate = 0.01
n_iter = 1000

#model
model = LogisticReg(13)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=l_rate)

for epoch in range(n_iter):
    
    #evaluate model
    y_pred = model(X_train)
    
    #calculate loss
    loss = criterion(y_pred, y_train)
    
    #calculate gradients
    loss.backward()
    
    #update gradients
    optimizer.step()
    
    #zero grad
    optimizer.zero_grad()
    
    if (epoch + 1) % 100 == 0:
        print(f'epoch: {epoch} loss: {loss.item()}')
    

epoch: 99 loss: 0.5545085668563843
epoch: 199 loss: 0.5423619747161865
epoch: 299 loss: 0.5313526391983032
epoch: 399 loss: 0.5213449001312256
epoch: 499 loss: 0.5122216939926147
epoch: 599 loss: 0.5038811564445496
epoch: 699 loss: 0.49623504281044006
epoch: 799 loss: 0.48920679092407227
epoch: 899 loss: 0.4827296733856201
epoch: 999 loss: 0.4767454266548157
epoch: 1099 loss: 0.47120335698127747
epoch: 1199 loss: 0.4660588502883911
epoch: 1299 loss: 0.46127280592918396
epoch: 1399 loss: 0.45681092143058777
epoch: 1499 loss: 0.4526427388191223
epoch: 1599 loss: 0.4487413465976715
epoch: 1699 loss: 0.44508299231529236
epoch: 1799 loss: 0.4416465163230896
epoch: 1899 loss: 0.4384128749370575
epoch: 1999 loss: 0.4353652894496918
epoch: 2099 loss: 0.43248865008354187
epoch: 2199 loss: 0.42976927757263184
epoch: 2299 loss: 0.4271949827671051
epoch: 2399 loss: 0.42475488781929016
epoch: 2499 loss: 0.4224388599395752
epoch: 2599 loss: 0.4202379584312439
epoch: 2699 loss: 0.4181439280509949
epo

In [50]:
with torch.no_grad():
  y_pred = model(X_test)
  y_pred_cls = y_pred.round()

  accuracy = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])

  print(f'Accuracy: {accuracy}')

Accuracy: 0.8681318759918213


### Resources

- Dataset taken from https://www.kaggle.com/ronitf/heart-disease-uci